In [2]:
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, Session
from sqlalchemy import String, insert, select, update

In [3]:
from CV_matcher.connection.PostgresConnection import PostgresConnection


In [4]:
pgConn = PostgresConnection("localhost", 54320, "postgres", "postgres", "postgres")

In [5]:
engine = pgConn.getEngine()

In [3]:
class Base(DeclarativeBase):
    pass

class TestModel(Base):
    __tablename__ = "test"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(50), nullable=True)

    def __repr__(self):
        return f"<TestModel(id={self.id}, name={self.name})>"

    def all_key_values(self, exclude_primary_key=False):
        if exclude_primary_key:
            return {c.name: getattr(self, c.name) for c in self.__table__.columns if not c.primary_key}
        return {c.name: getattr(self, c.name) for c in self.__table__.columns}

In [5]:
TestModel.__table__.name

'test'

In [16]:
TestModel.__table__.create(engine)

2024-08-20 16:22:11,043 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 16:22:11,043 INFO sqlalchemy.engine.Engine 
CREATE TABLE test (
	id SERIAL NOT NULL, 
	name VARCHAR(50), 
	PRIMARY KEY (id)
)


2024-08-20 16:22:11,044 INFO sqlalchemy.engine.Engine [no key 0.00053s] {}
2024-08-20 16:22:11,048 INFO sqlalchemy.engine.Engine COMMIT


In [17]:
# Insert into db, single records

TestModel(id=1, name="test1")

with Session(engine) as session:
    with session.begin():
        session.add(TestModel(id=2, name="test2"))
        session.add(TestModel(id=3, name="test3"))


2024-08-20 16:22:14,173 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 16:22:14,174 INFO sqlalchemy.engine.Engine INSERT INTO test (id, name) VALUES (%(id__0)s, %(name__0)s), (%(id__1)s, %(name__1)s)
2024-08-20 16:22:14,175 INFO sqlalchemy.engine.Engine [generated in 0.00009s (insertmanyvalues)] {'name__0': 'test2', 'id__0': 2, 'name__1': 'test3', 'id__1': 3}
2024-08-20 16:22:14,176 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
# Insert into db, bulk, list of Models

with Session(engine) as session:
    with session.begin():
        session.add_all(
            [
                TestModel(id=4, name="test4"), 
                TestModel(id=5, name="test5"), 
                TestModel(id=6, name="test6")
            ]
        )

2024-08-20 16:04:39,991 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 16:04:39,992 INFO sqlalchemy.engine.Engine INSERT INTO test (id, name) VALUES (%(id__0)s, %(name__0)s), (%(id__1)s, %(name__1)s), (%(id__2)s, %(name__2)s)
2024-08-20 16:04:39,993 INFO sqlalchemy.engine.Engine [cached since 1101s ago (insertmanyvalues)] {'name__0': 'test4', 'id__0': 4, 'name__1': 'test5', 'id__1': 5, 'name__2': 'test6', 'id__2': 6}
2024-08-20 16:04:39,994 INFO sqlalchemy.engine.Engine COMMIT


In [18]:
# Insert into db, bulk, list of dicts

with Session(engine) as session:
    with session.begin():
        session.execute(
            insert(TestModel),
            [
                {"id": 7, "name": "test7"},
                {"id": 8, "name": None},
                {"id": 9, "name": "test9"}
            ]
        )

2024-08-20 16:23:42,560 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 16:23:42,562 INFO sqlalchemy.engine.Engine INSERT INTO test (id, name) VALUES (%(id)s, %(name)s)
2024-08-20 16:23:42,562 INFO sqlalchemy.engine.Engine [generated in 0.00083s] {'id': 7, 'name': 'test7'}
2024-08-20 16:23:42,564 INFO sqlalchemy.engine.Engine INSERT INTO test (id) VALUES (%(id)s)
2024-08-20 16:23:42,565 INFO sqlalchemy.engine.Engine [generated in 0.00076s] {'id': 8}
2024-08-20 16:23:42,566 INFO sqlalchemy.engine.Engine INSERT INTO test (id, name) VALUES (%(id)s, %(name)s)
2024-08-20 16:23:42,566 INFO sqlalchemy.engine.Engine [cached since 0.004707s ago] {'id': 9, 'name': 'test9'}
2024-08-20 16:23:42,567 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
# Insert into db, bulk, list of dicts, with null

with Session(engine) as session:
    with session.begin():
        session.execute(
            insert(TestModel),
            [
                {"id": 10, "name": "test10"},
                {"id": 11},
                {"id": 12, "name": "test12"}
            ]
        )

2024-08-20 16:24:47,391 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 16:24:47,392 INFO sqlalchemy.engine.Engine INSERT INTO test (id, name) VALUES (%(id)s, %(name)s)
2024-08-20 16:24:47,393 INFO sqlalchemy.engine.Engine [cached since 64.83s ago] {'id': 10, 'name': 'test10'}
2024-08-20 16:24:47,394 INFO sqlalchemy.engine.Engine INSERT INTO test (id) VALUES (%(id)s)
2024-08-20 16:24:47,394 INFO sqlalchemy.engine.Engine [cached since 64.83s ago] {'id': 11}
2024-08-20 16:24:47,395 INFO sqlalchemy.engine.Engine INSERT INTO test (id, name) VALUES (%(id)s, %(name)s)
2024-08-20 16:24:47,395 INFO sqlalchemy.engine.Engine [cached since 64.83s ago] {'id': 12, 'name': 'test12'}
2024-08-20 16:24:47,397 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
# Select all records from db

with Session(engine) as session:
    with session.begin():
        statement = select(TestModel)
        testModels = session.scalars(statement).all()
        print(testModels[0])

2024-08-21 15:33:28,904 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-21 15:33:28,905 INFO sqlalchemy.engine.Engine SELECT test.id, test.name 
FROM test
2024-08-21 15:33:28,906 INFO sqlalchemy.engine.Engine [cached since 473.7s ago] {}
<TestModel(id=2, name=test2)>
2024-08-21 15:33:28,907 INFO sqlalchemy.engine.Engine COMMIT


In [37]:
# Update one record

updateModel = TestModel(id=3, name="test3_updated_2")

with Session(engine) as session:
    with session.begin():
        session.execute(
            update(TestModel)
            .where(TestModel.id == updateModel.id)
            .values(updateModel.all_key_values(exclude_primary_key=True))
        )
        statement = select(TestModel).where(TestModel.id == 3)
        testModels = session.scalars(statement).all()
        print(testModels[0])

{'id': 3, 'name': 'test3_updated_2'}
